In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import datetime
import time
import os
import pathlib
import matplotlib.pyplot as plt
 
# Some stuff to make utils-function work
import sys
sys.path.append('../utils')
from pipeline import create_dataset, split_and_create_dataset, prepare_for_training
from create_model import create_model, create_callbacks, get_class_weights
from utils import show_image, class_distribution, write_to_file
from utils import print_split_info, unpipe, tf_bincount, checkout_dataset
%load_ext autoreload
%autoreload 2

# Jupyter-specific
%matplotlib inline

project_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

Some parameters

In [ ]:
data_dir = pathlib.Path('/home/henriklg/master-thesis/data/hyper-kvasir/labeled_ttv/')
unlab_dir = pathlib.Path('/home/henriklg/master-thesis/data/hyper-kvasir/unlabeled_ttv/')

iteration = 0
dir_name = str(iteration)+"_teacher"
log_dir = "./logs/{}/{}".format(project_time, dir_name)

conf = {
    # Dataset
    "data_dir": data_dir,
    "unlab_dir": unlab_dir,
    "ds_info": 'hypkva',
    "augment": ["crop","flip","brightness","saturation","contrast","rotate"],
    "aug_mult": 0.7,
    "resample": False,
    "class_weight": True,
    "shuffle_buffer_size": 2000,        # 0=no shuffling
    "seed": 2511,
    "neg_class": None,                 # select neg class for binary ds (normal class)
    "outcast": None,                   # list of folders to drop - currently only works for 1 item
    # Model
    "model": 'EfficientNetB0',
    "weights": "imagenet",                   # which weights to initialize the model with
    "dropout": 0.2,
    "num_epochs": 4,
    "batch_size": 16,
    "img_shape": (128, 128, 3),
    "learning_rate": 0.001,
    "optimizer": 'Adam',
    "final_activation": 'softmax',     # sigmoid for binary ds
    # Callbacks
    "tensorboard": False,
    "learning_schedule": False,
    "decay_rate": 0,                   # 128:0.25   64:1.0   32:4.0   16:16   8:64
    "checkpoint": False,
    "early_stopp": False,
    "early_stopp_patience": 7,
    # Misc
    "verbosity": 0,
    "keep_threshold": 0.0,
    "dir_name": dir_name,
    "log_dir": log_dir,
    "cache_dir": "./cache",
    }

Create training, testing and validation dataset from utils/data_prep.py.  
Returns tf.dataset for shuffled, cached and batched data

In [ ]:
ds = create_dataset(conf)

# Full EfficientNetB0

In [ ]:
from efficientnet.tfkeras import EfficientNetB0 as EfficientNet
from tensorflow.keras import layers
from tensorflow.python.keras.models import Sequential, Model, load_model

In [ ]:
def build_model(output_size, input_shape=(224, 224, 3), final_activation="softmax"):

    base_model = EfficientNet(
        input_shape=input_shape,
        weights="imagenet",
        include_top=False
    )
    
    base_model.trainable = True
    
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(conf["dropout"])(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(conf["dropout"])(x)
    
    output = layers.Dense(output_size, activation=final_activation)(x)
    
    return Model(outputs=output, inputs=base_model.input)

In [ ]:
model = build_model(
    output_size = conf["num_classes"],
    input_shape = conf["img_shape"],
    final_activation = conf["final_activation"]
)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=conf["learning_rate"]),
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy']
)

In [ ]:
model.summary()

In [ ]:
# Save image of layers
try:
    tf.keras.utils.plot_model(model, '{}/{}.pdf'.format(conf["log_dir"], "entire_model_noshapes"), show_shapes=False)
except ValueError:
    pass

# Model with no top

In [ ]:
teacher_model = create_model(conf)

In [ ]:
# Save image of layers
try:
    tf.keras.utils.plot_model(teacher_model, '{}/{}.pdf'.format(conf["log_dir"], "my_model"), show_shapes=True)
except ValueError:
    pass